In [41]:
import requests
import json
import re
from bs4 import BeautifulSoup as bs4
from IPython.display import display, HTML
import os


In [42]:
client_id = os.environ.get("DEVIANTART_CLIENT_ID")
client_secret = os.environ.get("DEVIANTART_CLIENT_SECRET")

In [43]:
token_request = requests.post("https://www.deviantart.com/oauth2/token", params= {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "client_credentials"
} )

In [44]:
token = token_request.json()["access_token"]

In [45]:
esk_request = requests.get("https://www.deviantart.com/api/v1/oauth2/gallery/all", 
headers={
    'Authorization': 'Bearer {}'.format(token)
    }, 
    params= {
    "username": "esk-masterlist",
    "offset": 250,
    "limit": 24
}).json()["results"]

In [46]:
deviation_ids = [esk_request[i]["deviationid"] for i in range(0,24)]
print(deviation_ids)

['50D852A7-7111-765D-7791-F16442AD9FB7', 'D95C613D-2A87-8539-5FCB-D434CF72EF13', 'A6E1DC2C-84ED-CF87-C8F0-45C5F2B4944C', '706D13C6-8AD1-9EF3-81E9-5295A2212B26', '93671489-9E2C-DC76-B595-DCFCCEEBB1AE', 'D5E62D64-729F-667B-4494-405D194C6436', '685CBC73-A17A-6F24-1E45-57C3B53396E4', '637981FA-4E5F-0F69-1588-CA33932021DF', 'C5E90B48-1077-F425-839D-8E6D3ABF4F4E', '595BBF5B-056C-77FD-206C-C95A3F803D02', 'BB5E44F9-0D95-9298-FC81-F55330EA725E', '8F8E12C4-2D59-3849-7E03-B04BC25D7D2C', '88C226BC-439E-890E-B1F8-1C2F564F354E', '1781BA25-EE3D-156E-FF33-B36C5949FD1D', '2CF161BE-8763-56E3-AB13-B4C302B6B49C', 'C6307457-11DE-3CD2-FF99-C1566191E4DA', 'AEFE9918-B6CB-A21B-217B-64D91CAE4544', 'C0B6CEBE-EAE7-43CD-CFEB-082A49B5AF49', '72C8A3DF-E66F-5960-C843-F53958CEBEE1', '86557745-8CAF-689C-4C78-A35867660DE3', '75176685-1F88-D48E-695D-249AE7B5A116', '2200D7E2-1383-4925-D7AB-3ED89FFFD108', '3AD83C89-1640-D48D-3D3B-416621938993', '7A2A7411-D544-7FBD-FE13-740D161E515C']


In [47]:
esk_request = requests.post("https://www.deviantart.com/api/v1/oauth2/deviation/metadata", 
    data={
        "deviationids[]": deviation_ids
    },
    headers={
        'Authorization': 'Bearer {}'.format(token)
    }).json()["metadata"]

In [113]:
def esk_record_from_web_response(raw_esk_record):
    parsed_html = bs4(raw_esk_record["description"])
    bold_texts = parsed_html.body.select('div > b', class_="legacy-journal")
    for bold in bold_texts:
        if re.fullmatch(r"\s+", bold.string) != None:
            continue
        if bold.string == "designer":
            continue
        if bold.string == "ownership":
            break
        tmp = bold.next_sibling.strip()
        if str.endswith(tmp, ".") or str.endswith(tmp, "("):
            tmp = tmp[:-1].strip()
        #if "mutation" in bold.string:    
        tmp = re.sub(r"\(.*\)", "", tmp)
        raw_esk_record[bold.string] = tmp
    raw_esk_record.pop("description", None)
    owner_history = parsed_html.body.find('b', string= "owner history", recursive=True)
    raw_esk_record["owner count"] = len(owner_history.find_next_sibling("sub").findChildren("br")) + 1
    return raw_esk_record


In [114]:
esk_descriptions = [{"title": esk["title"], "description": esk["description"]} for esk in esk_request if (re.fullmatch( r"\d{3,4}", esk["title"]) != None)]
for esk in esk_descriptions:
    print(esk_record_from_web_response(esk))



{'title': '5458', 'origin': 'trespasser', 'nature': 'spiteful', 'boundary': 'dusty road', 'size': 'fleeting', 'species': 'esk', 'collection': 'MYO', 'uncommon traits': 'false ears, socks, claws, tail-tip marking, spots', 'rare traits': 'unusual coverage', 'nature features': 'maidenstears', 'accessories': 'wooden toy', 'owner count': 1}
{'title': '5457', 'origin': 'abnormal', 'nature': 'eerie', 'boundary': 'haunted mirror maze', 'size': 'uncanny', 'species': 'esk', 'collection': 'MYO', 'uncommon traits': 'claws, stripes, dark mask, unique markings', 'rare traits': 'manx, bright eye', 'nature features': 'thorny branches', 'mutations': 'animal trait , animated nature feature', 'original form': 'lost toy', 'accessories': 'stained veil', 'enchantments': 'mirrorscape', 'owner count': 1}
{'title': '5456', 'origin': 'traveler', 'nature': 'delightful', 'boundary': 'coastal waters', 'size': 'perfect', 'species': 'esk', 'collection': 'MYO', 'uncommon traits': 'dark mask, spots, bold markings, eye